In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import shapefile as sf
from pyproj import Proj
from censusgeocode import CensusGeocode

import datetime

%matplotlib inline

In [2]:
crime_reports = pd.read_csv('ARJISPublicCrime091416.csv')

In [3]:
crime_reports.columns.values

array(['agency', 'Charge_Description_Orig', 'activityDate',
       'BLOCK_ADDRESS', 'ZipCode', 'community'], dtype=object)

In [3]:
crime_reports['BLOCK_ADDRESS'] = crime_reports['BLOCK_ADDRESS'].replace({'BLOCK': ''}, regex=True)

In [4]:
cg = CensusGeocode()

In [5]:
crime_reports['lat'] = None
crime_reports['lon'] = None 
crime_reports['tract'] = None
crime_reports['blockgroup'] = None 
crime_reports['block '] = None

In [6]:
crime_reports = pd.read_csv('output_files/crime_reports_prelim.csv')

/Users/kschles/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
start = datetime.datetime.now()

print len(crime_reports)

for i in range(58100,len(crime_reports)): 
    address1 = crime_reports.loc[i, 'BLOCK_ADDRESS']
    n_attempts = 0
    while True: 
        try: 
            n_attempts += 1
            temp = cg.address(address1, city=crime_reports.loc[i, 'community'], state='CA')
            break 
        except: 
            if n_attempts>=5: 
                print 'bad in try1 ' + address1
                break 
    if (temp!=[]): 
        if (len(temp)>0): 
            crime_reports.loc[i, 'lon'] = temp[0]['coordinates']['x']
            crime_reports.loc[i, 'lat'] = temp[0]['coordinates']['y']
            
            if ('status' not in temp[0]['geographies']['2010 Census Blocks'][0].keys()[0]): 
                crime_reports.loc[i, 'tract'] = temp[0]['geographies']['2010 Census Blocks'][0]['TRACT']
                crime_reports.loc[i, 'block'] = temp[0]['geographies']['2010 Census Blocks'][0]['BLOCK']
                crime_reports.loc[i, 'blockgroup'] = temp[0]['geographies']['2010 Census Blocks'][0]['BLKGRP']

    if (i % 100 == 0): 
        finish = datetime.datetime.now()
        print i, 'Time Diff: ', finish-start, 'Finish Time: ', finish
        start = datetime.datetime.now()

    if (i % 250 == 0): 
        crime_reports.to_csv('output_files/crime_reports_prelim.csv', index=False)

66304
58100 Time Diff:  0:00:07.402313 Finish Time:  2016-09-20 00:29:17.926646
58200 Time Diff:  0:01:11.989915 Finish Time:  2016-09-20 00:30:29.916768
58300 Time Diff:  0:01:01.869257 Finish Time:  2016-09-20 00:31:31.786263
58400 Time Diff:  0:01:17.266027 Finish Time:  2016-09-20 00:32:49.052595
58500 Time Diff:  0:01:16.159653 Finish Time:  2016-09-20 00:34:05.212498
58600 Time Diff:  0:01:11.897122 Finish Time:  2016-09-20 00:35:17.110021
58700 Time Diff:  0:01:15.474591 Finish Time:  2016-09-20 00:36:32.584967
58800 Time Diff:  0:01:02.981584 Finish Time:  2016-09-20 00:37:35.566770
58900 Time Diff:  0:01:28.841430 Finish Time:  2016-09-20 00:39:04.408404
59000 Time Diff:  0:01:16.410283 Finish Time:  2016-09-20 00:40:20.818957
59100 Time Diff:  0:01:27.655941 Finish Time:  2016-09-20 00:41:48.475300
59200 Time Diff:  0:01:10.715987 Finish Time:  2016-09-20 00:42:59.191513
59300 Time Diff:  0:01:07.888346 Finish Time:  2016-09-20 00:44:07.080166
59400 Time Diff:  0:01:06.015013

In [387]:
print i
print address1

21833
1300   PARK BOULEVARD


In [8]:
crime_reports.to_csv('output_files/crime_reports_prelim.csv', index=False)

Organise arrests by tract and block group

In [31]:
crime_reports['arrests_count'] = 1.0

In [28]:
crime_reports['tract'] = crime_reports['tract'].astype(float)
crime_reports['blockgroup'] = crime_reports['blockgroup'].astype(float)

In [39]:
pa_by_bg = crime_reports[['tract', 'blockgroup', 'pa_count']].groupby(['tract','blockgroup'], as_index=False).agg('count')

In [40]:
#cpa_by_bg.to_csv('output_files/crime_reports_bg.csv', index=False)